In [7]:
# from IPython.display import clear_output
# !pip install ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git
# !pip install scipy
# clear_output()

In [1]:
import torch
import clip
from PIL import Image
import numpy as np
import glob
import math
from scipy import spatial
import os

/home/nhattuong/anaconda3/envs/clip/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Check Features

In [3]:
def calculate_cosine_distance(a, b):
    cosine_distance = float(spatial.distance.cosine(a, b))
    return cosine_distance

def calculate_cosine_similarity(a, b):
    cosine_similarity = 1 - calculate_cosine_distance(a, b)
    return cosine_similarity

In [43]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

######### Load Features #########
feats_path = '/home/hoangtv/Desktop/OS/ubuntu/Dataset_AIC2022/CLIPFeatures_C00_V00/C00_V0000.npy'
feats = np.load(feats_path)
##################################

##################################
distance = []
re_feats = []

keyframe_paths = glob.glob('/home/hoangtv/Desktop/OS/ubuntu/Dataset_AIC2022/KeyFramesC00_V00/C00_V0000/*.jpg')
keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

for idx, keyframe_path in enumerate(keyframe_paths):
  feats_idx = feats[idx].astype(np.float32)
  image = preprocess(Image.open(keyframe_path)).unsqueeze(0).to(device)

  with torch.no_grad():
      image_feats = model.encode_image(image)

  image_feats /= image_feats.norm(dim=-1, keepdim=True)
  image_feats = image_feats.detach().cpu().numpy().astype(np.float32).flatten()

  re_feats.append(image_feats)

  # Tính similarity
  score = calculate_cosine_similarity(image_feats, feats_idx)
  distance.append(score)

## Re Features

In [4]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/16", device=device)

################# Config Folder #################
des_path = '/media/nhattuong/Data/Exact_CLIP_Features/CLIP_Features/CLIPFeatures_C02_V03/'
paths = '/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03'
#################################################

video_paths = sorted(glob.glob(f"{paths}/*/"))
video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

# os.makedirs(des_path, exist_ok=True)

for vd_path in video_paths:
  print(vd_path)
  # check_file = int(vd_path.split('/')[-1].replace('C02_V',''))
  # if check_file <= 349:
  #   print(f"Skip: {vd_path}")
  #   continue

  re_feats = []
  keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
  keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

  for keyframe_path in keyframe_paths:
    image = preprocess(Image.open(keyframe_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        image_feats = model.encode_image(image)

    image_feats /= image_feats.norm(dim=-1, keepdim=True)
    image_feats = image_feats.detach().cpu().numpy().astype(np.float16).flatten() 

    re_feats.append(image_feats)

  name_npy = vd_path.split('/')[-1]
  outfile = f'{des_path}/{name_npy}.npy'
  np.save(outfile, re_feats)

cuda
/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0300
Skip: /media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0300
/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0301
Skip: /media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0301
/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0302
Skip: /media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0302
/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0303
Skip: /media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0303
/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0304
Skip: /media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0304
/media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0305
Skip: /media/nhattuong/Data/TranseNet/TransNet_Database/KeyFramesC02_V03/C02_V0305
/media/nhattuong/Data/TranseNet/Tra